<a href="https://colab.research.google.com/github/stephenfrein/csc8490/blob/main/ExternalApplicationtoOracle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Basic Connection and Operations

In [ ]:
# PIP is Python Improved Packaging
# command below installs a Python code package that allows to connect to Oracle easily
# the ! means "run this from the command line rather than as a python command"
!pip install oracledb

In [ ]:
# import allows us to load an available package into memory so we can use it
import oracledb

# set up our connection information
db_user = "demo_user"
db_password = "demo_pass"
connect_string = "vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL"

# make a connection - this creates a session with the database
conn = oracledb.connect(user=db_user, password=db_password, dsn=connect_string)

# prove that we've made the connection
print("Database version:", conn.version)

In [ ]:
# select some data from the database
# a cursor in this case is similar to a PL/SQL cursor - a memory area used for processing SQL
cursor = conn.cursor()
# for each iteration of the for loop, result gets populated with the next row from the query results
for result in cursor.execute("select * from HR.Countries"):
    print(result)


In [ ]:
# calling a stored procedure
# run the stored procedure setup in the slides first
# set up a variable to accept the procedure output
result = cursor.var(int)
# calling the procedure
cursor.callproc("DOUBLER", [6, result])
# show result
print(result)
# show just the value frrom result
print(result.getvalue())

In [ ]:
# insert some records
# assumes you've created the ExtAppsTest table using the slides
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (1, 'Blue', 14)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (2, 'Red', 16)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (3, 'Blue', 19)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (4, 'Yellow', 20)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (5, 'Red', 22)")



In [ ]:
# see the data you just entered
for result in cursor.execute("select * from ExtAppsTest"):
    print(result)

In [ ]:
# update one of the records
cursor.execute("update ExtAppsTest set TextCol = 'Purple' where IdVal = 5")

In [ ]:
for result in cursor.execute("select * from ExtAppsTest"):
    print(result)

In [ ]:
# make changes public and permanent
conn.commit()

In [ ]:
# clears out records
cursor.execute("delete from ExtAppsTest")

In [ ]:
# close both the cursor and the connection to free up resources
cursor.close()
conn.close()

Notes:

Connect and query
SQL injection - variables
Secrets
Stored procedures
Bind variables
Connection pooling
error handling

# Stage 2: Don't Keep Secrets in Your Code

In [ ]:
# condensed example of code we looked at in Stage 1
import oracledb
# set up our connection information
db_user = "demo_user"
db_password = "demo_pass"
connect_string = "vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL"
# make a connection - this creates a session with the database
conn = oracledb.connect(user=db_user, password=db_password, dsn=connect_string)
# select some data from the database
cursor = conn.cursor()
for result in cursor.execute("select * from HR.Countries where region_id = 1"):
    print(result)

This is bad because we have username and password details directly in our code. Anybody with access to the source code can now login to the database with the exposed credentials. We need to separate those from the code.

In [ ]:
!cat > /path/to/newFile.text<< EOF
username=demo_user
password=demo_pass
EOF

In [ ]:
!pwd

# Stage 2: Example of A Web App That Executes Queries Based on Parameter

In [ ]:
    from google.colab import output
    output.serve_kernel_port_as_window(8084)

In [ ]:
    from flask import Flask
    from flask import jsonify
    from flask import request
    import oracledb
    app = Flask(__name__)

    @app.route("/")
    def hello():
        return "Hello, World!"

    @app.route("/query")
    def query():
        id = request.args.get('id')
        # set up our connection information
        db_user = "demo_user"
        db_password = "demo_pass"
        connect_string = "vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL"

        # make a connection - this creates a session with the database
        conn = oracledb.connect(user=db_user, password=db_password, dsn=connect_string)
        # create cursor for SQL processing
        cursor = conn.cursor()
        # execute a query using the parameter passed in
        cursor.execute("select * from hr.employees where employee_id = " + str(id))
        result = cursor.fetchone()
        # close both the cursor and the connection to free up resources
        cursor.close()
        conn.close()

        return str(result)

    if __name__ == "__main__":
        app.run(host='0.0.0.0', port=8084)

In [ ]:
# insert some records
# assumes you've created the ExtAppsTest table using the slides
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (1, 'Blue', 14))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (2, 'Red', 16))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (3, 'Blue', 19))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (4, 'Yellow', 20))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (5, 'Red', 22))



In [ ]:
# update one of the records
cursor.execute("update ExtAppsTest set TextCol = :1 where IdVal = :2", ('Purple', 5))

In [ ]:
# can we pass in a column dynamically?
cursor.execute("update ExtAppsTest set :1 = :2 where IdVal = :3", ("TextCol",'Magenta', 5))